In [ ]:
BITSTREAM_PATH = '../overlay/demo.bit'
BUFFER_SIZE_W = 16

In [ ]:
import os
import time
import ipywidgets as widgets
from pynq import allocate
from IPython.display import display
from axi.driver import SimpleAxiMasterDriver

In [ ]:
if not os.path.exists(BITSTREAM_PATH):
    raise FileNotFoundError(f'Bitstream not found at {BITSTREAM_PATH}. Run \'make bitstream\'.')

driver = SimpleAxiMasterDriver(BITSTREAM_PATH)

In [ ]:
buffer = allocate(shape=(BUFFER_SIZE_W,), dtype='u4')
buffer_addr = buffer.physical_address

buffer[:] = [0] * BUFFER_SIZE_W
buffer.flush()

In [ ]:
def hex_dump(buffer, num_bytes=64, row_size=8):
    buffer.invalidate()
    
    lines = []
    header = f"{'Offset':<8} | {'00 01 02 03 04 05 06 07':<23}"
    lines.append(header)
    lines.append("-" * len(header))
    
    byte_list = []
    limit = min(num_bytes // 4, len(buffer))
    for i in range(limit):
        val = buffer[i]
        byte_list.extend([val & 0xFF, (val >> 8) & 0xFF, (val >> 16) & 0xFF, (val >> 24) & 0xFF])

    for i in range(0, len(byte_list), row_size):
        chunk = byte_list[i:i+row_size]
        hex_strs = [f"{b:02X}" for b in chunk]
        row_str = " ".join(hex_strs)
        lines.append(f"+{hex(i):<7} | {row_str}")
        
    return "\n".join(lines)

In [ ]:
# ----------------------------------------------------------------------
# Config
# ----------------------------------------------------------------------
ADDR_LEDS     = 0x40000000
ADDR_SWITCHES = 0x40000008

# ----------------------------------------------------------------------
# UI Elements
# ----------------------------------------------------------------------

w_addr = widgets.Text(value=hex(buffer.physical_address), description="Addr:", layout={'width': '180px'})
w_data = widgets.Text(value="0x11223344", description="Data:", layout={'width': '180px'})
w_size = widgets.Dropdown(
    options=[('Byte (8b)', 0), ('Half (16b)', 1), ('Word (32b)', 2)],
    value=2, description="Size:", layout={'width': '180px'}
)
w_read_result = widgets.Text(value="", description="Read Val:", disabled=True, layout={'width': '250px'})
btn_mem_write = widgets.Button(description="Write", button_style='warning')
btn_mem_read  = widgets.Button(description="Read", button_style='success')
btn_clear     = widgets.Button(description="Clear", button_style='danger')
btn_reset     = widgets.Button(description="Reset", button_style='danger')

btn_led0 = widgets.ToggleButton(value=False, description="BLUE", button_style='', layout={'width':'80px'})
btn_led1 = widgets.ToggleButton(value=False, description="RED", button_style='', layout={'width':'80px'})
btn_led2 = widgets.ToggleButton(value=False, description="GREEN", button_style='', layout={'width':'80px'})
btn_write_leds = widgets.Button(description="Set LEDs", button_style='primary')

lbl_switches = widgets.Label(value="Switches: [ ? ? ]", layout={'width': '200px', 'font_weight': 'bold'})
btn_read_sw  = widgets.Button(description="Read Switches", button_style='info')

w_cpu_val = widgets.Text(value="0xAABBCCDD", description="CPU Val:", layout={'width': '180px'})
w_cpu_idx = widgets.BoundedIntText(value=0, min=0, max=63, description="Word Idx:", layout={'width': '150px'})
btn_cpu_write = widgets.Button(description="CPU Write", button_style='')
btn_cpu_zero  = widgets.Button(description="Zero Buffer", button_style='')

out_buffer = widgets.Output(layout={'border': '1px solid #444', 'padding': '10px', 'font_family': 'monospace'})
out_log    = widgets.Output(layout={'border': '1px solid #ccc', 'height': '150px', 'overflow_y': 'scroll'})

# ----------------------------------------------------------------------
# Logic
# ----------------------------------------------------------------------

def log(msg):
    with out_log:
        print(msg)

def refresh_buffer_view():
    out_buffer.clear_output()
    with out_buffer:
        print(f"Buffer Base: {hex(buffer.physical_address)}")
        print(hex_dump(buffer, num_bytes=64))

def on_mem_write(b):
    try:
        addr = int(w_addr.value, 16)
        data = int(w_data.value, 16)
        size = w_size.value
        log(f"MEM WRITE -> Addr:{hex(addr)} Data:{hex(data)}")
        res = driver.write(size, addr, data)
        log(f"  Finished in {res.latency} cycles")
        if res.status.value != 'done':
            log(f"  Warning: Status is {res.status.value.upper()}")
        refresh_buffer_view()
    except Exception as e: log(f"ERR: {e}")

def on_mem_read(b):
    try:
        addr = int(w_addr.value, 16)
        size = w_size.value
        log(f"MEM READ  <- Addr:{hex(addr)}")
        res = driver.read(size, addr)
        log(f"  Finished in {res.latency} cycles")
        w_read_result.value = f"{hex(res.data)} ({res.status.value.upper()})"
        log(f"  Data: {hex(res.data)}")
    except Exception as e: log(f"ERR: {e}")


def on_clear(b):
    log("Clearing status...")
    driver.clear()
    log("Done.")
        
def on_reset(b):
    log("Resetting PL...")
    driver.hard_reset()
    log("Done.")

def on_write_leds(b):
    val = 0
    val |= (1 if btn_led0.value else 0) << 0
    val |= (1 if btn_led1.value else 0) << 1
    val |= (1 if btn_led2.value else 0) << 2
    
    log(f"DEV WRITE -> LEDs ({hex(ADDR_LEDS)}) Val:{bin(val)}")
    driver.write_word(ADDR_LEDS, val)

def on_read_sw(b):
    log(f"DEV READ  <- Switches ({hex(ADDR_SWITCHES)})")
    res = driver.read_word(ADDR_SWITCHES)
    
    sw0 = (res.data >> 0) & 1
    sw1 = (res.data >> 1) & 1
    lbl_switches.value = f"Switches: [ {sw1} {sw0} ]"
    log(f"  Raw: {hex(res.data)} | SW1={sw1}, SW0={sw0}")

def on_cpu_write(b):
    try:
        val, idx = int(w_cpu_val.value, 16), w_cpu_idx.value
        buffer[idx] = val
        buffer.flush()
        refresh_buffer_view()
    except: pass

def on_cpu_zero(b):
    for i in range(len(buffer)): buffer[i] = 0
    buffer.flush()
    refresh_buffer_view()

# ----------------------------------------------------------------------
# Callbacks
# ----------------------------------------------------------------------
btn_mem_write.on_click(on_mem_write)
btn_mem_read.on_click(on_mem_read)
btn_clear.on_click(on_clear)
btn_reset.on_click(on_reset)
btn_write_leds.on_click(on_write_leds)
btn_read_sw.on_click(on_read_sw)
btn_cpu_write.on_click(on_cpu_write)
btn_cpu_zero.on_click(on_cpu_zero)

def update_btn_color(change):
    change['owner'].button_style = 'success' if change['new'] else ''
for b in [btn_led0, btn_led1, btn_led2]:
    b.observe(update_btn_color, 'value')

# ----------------------------------------------------------------------
# Layout
# ----------------------------------------------------------------------
refresh_buffer_view()

tab_mem = widgets.VBox([
    widgets.HBox([w_addr, w_data, w_size]),
    widgets.HBox([btn_mem_write, btn_mem_read, btn_clear, btn_reset]),
    widgets.HBox([w_read_result]),
    widgets.HTML("<hr>"),
    widgets.Label("Buffer View:"),
    out_buffer,
    widgets.HBox([w_cpu_idx, w_cpu_val, btn_cpu_write, btn_cpu_zero])
])

tab_dev = widgets.VBox([
    widgets.HTML("<b>LED Control (Addr: 0x40000000)</b>"),
    widgets.HBox([btn_led0, btn_led1, btn_led2]),
    btn_write_leds,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>Switch Status (Addr: 0x40000008)</b>"),
    widgets.HBox([btn_read_sw, lbl_switches])
])

tabs = widgets.Tab(children=[tab_mem, tab_dev])
tabs.set_title(0, 'Memory')
tabs.set_title(1, 'Devices')

ui = widgets.VBox([
    widgets.HTML("<h3>AXI Master Demo</h3>"),
    tabs,
    widgets.HTML("<h4>Log</h4>"),
    out_log
])

# Using the Demo

The cell below draws the UI prepared in the previous step. It enables you to write to and read from a buffer of fixed size, as well as turn on and off RGB LEDs and read switches. You can do that from the memory and devices tabs, respectively.

## Memory

You interact with memory from the Memory tab. You can write to and read from the buffer at the address shown in the Address field and in the buffer view (`Buffer Base`).

A hex dump of the buffer is shown. Each line is a 64-bit double word, displayed as little-endian.

Transaction information will be shown in the log. That includes data, status, and latency (measured as cycles from start to end of transaction at 100MHz).

The four green LEDs show the current state of the AXI Master.

* **All off**: Idle, status cleared.
* **`LD0` on**: Idle, finished without error.
* **`LD1` on**: Idle, finished with device error.
* **`LD0` and `LD1` on**: Idle, tried to access invalid address. This probably means misaligned data.
* **Other:** Transaction in progress. You will probably not see this, things happen fast.

You can clear the status at any point by clicking the `Clear` button.

The `Reset` button sends a reset signal to the Master and GPIO module of the devices.

**Writing to memory:**

1. Enter the address you want to write to in the `Addr` field.
2. Enter the data you want to write in the `Data` field.
3. Select the size of the data you want to write from the `Size` field.
4. Click the `Write` button.

> **Note:** The address must be aligned to the size specified by the size, i.e. if you are writing a half-word, the address must be aligned to a half-word (16-bit) boundry (least-significant-bit must be 0) etc.

You will see the data in the Buffer View.

**Reading from memory:**

1. Enter the address you want to read from in the `Addr` field.
2. Select the size of the data you want to read from the `Size` field.
3. Click the `Read` button.

The data will be shown in the `Read Val` field.

## Devices

You can control RGB LEDs and read the two switches next to them.

**Setting LEDs**

1. Select which LEDs you want on by clicking the corresponding buttons (`BLUE`, `RED`, or `GREEN`).
2. Click `Set LEDs`.

**Reading switch status:**

1. Set the switches in the desired position.
2. Click `Read Switches`.

Status will be shown next to the button, with the left element corresponding to the left switch, and the rigth one to the right.

In [ ]:
display(ui)